In [1]:
import requests

In [75]:
#election_years = range(1788,2024,4)
election_years = [1788]
#election_years = [1788, 1792, 1796]

In [78]:
def get_election_info(year):
    params = {
        "action":"parse",
        "page": str(year) + " United States presidential election",
        "prop": "wikitext",
        "section": 0,
        "format": "json",
    }
    
    url = "https://en.wikipedia.org/w/api.php"
    
    if year == 1788:
        year = str(year) + "-89"
        url += "?"
        for label, param in params.items():
            url += (label + "=" + str(param).replace(" ", "+") + "&")
        url += "redirects"
        resp = requests.get(url)
    else:
        resp = requests.get(url, params=params)
        
    data = resp.json()
    wikitext=data['parse']['wikitext']['*']
    start_ind = wikitext.find('Infobox election')
    end_ind = wikitext.find('map')
    wikitext = wikitext[start_ind:end_ind]
    
    wikitext = wikitext.replace('[[', '').replace(']]', '').replace('\'\'\'', '').replace("<!--", "").replace("-->", "")
    nominee_num = 1
    lines = wikitext.split('\n|')

    print(wikitext)
    
    #list of dictionaries
    nominees = [] #list of nominees
    nominees_dict = {} #nominee info
    
    for line in lines: 
        line = line.strip()
        if line.startswith('turnout'):
            turnout_str = line.split('=')[1].split('%')[0].strip()
            if turnout_str.find('\n') > 0:
                turnout = turnout_str.split('\n')[1]
            else:
                turnout = turnout_str
            nominees_dict['turnout'] = turnout
        elif line.startswith('needed_votes'):
            needed_votes = line.split('=')[1][1:]
            nominees_dict['needed_votes'] = needed_votes
        elif line.startswith('nominee'):
            nominee = line.split('=')[1].split('\n')[0].strip()
            nominees_dict['nominee']=nominee
        elif line.startswith('party'):
            party_str = line.split('=')[1][1:]
            party = party_str.split("(")[0].strip()
            nominees_dict['party']=party
        elif line.startswith('running_mate'):
            running_mate = line.split('=')[1][1:].strip()
            nominees_dict['running_mate']=running_mate
        elif line.startswith('home_state'):
            home_state_str = line.split('=')[1][1:]
            home_state = home_state_str.split("(")[0].split("{{")[0].strip()
            nominees_dict['home_state']=home_state
        elif line.startswith('electoral_vote'):
            electoral_vote_str = line.split('=')[1][1:]
            electoral_vote = electoral_vote_str.split("{{")[0].strip()
            nominees_dict['electoral_vote']=electoral_vote
        elif line.startswith('states_carried'):
            states_carried_str = line.split('=')[1][1:]
            states_carried = states_carried_str.split("+")[0].strip()
            nominees_dict['states_carried']=states_carried
        elif line.startswith('popular_vote'):
            popular_vote_str = line.split('=')[1].split("<")[0]
            popular_vote = popular_vote_str.split("{{")[0].strip()
            nominees_dict['popular_vote']=popular_vote
        elif line.startswith('percentage'):
            percentage_line_str = line.split('=')[1]
            percentage_line = percentage_line_str.split("%")[0].strip()
            if percentage_line.startswith("{{"):
                percentage_lineparts = percentage_line.split('|')
                total = percentage_lineparts[2].replace("TOTAL:", "").replace(",", "").strip()
                popular_vote = nominees_dict['popular_vote'].replace(",", "")
                percentage = str(round((float(popular_vote) / float(total)) * 100, 1))
            else:
                percentage = percentage_line
        
            nominees_dict['percentage'] = percentage
    
            #Reset the dictionary because percentage is the last field
            nominees.append(nominees_dict)
            turnout = nominees_dict['turnout']
            needed_votes = nominees_dict['needed_votes']
            nominees_dict = {}
            nominees_dict['turnout']=turnout
            nominees_dict['needed_votes']=needed_votes
    return nominees

In [79]:
election_info = {year:get_election_info(year) for year in election_years}
election_info

Infobox election
| election_name = 1788–89 United States presidential election
| country = United States
| flag_year = 1777
| type = presidential
| ongoing = no
| previous_election = 
| next_election = 1792 United States presidential election
| next_year = 1792
| votes_for_election = 69 members of the United States Electoral College|Electoral College
| needed_votes = at least 35 electoral
| turnout = 11.6%<ref>{{cite web|url=http://www.electproject.org/national-1789-present|title=National General Election VEP Turnout Rates, 1789-Present|work=United States Election Project|publisher=CQ Press}}</ref>
| election_date = {{start and end dates|1788|12|15|1789|1|10}}
| image1 = Gilbert Stuart Williamstown Portrait of George Washington.jpg
| nominee1 = George Washington
| party1 = Independent (United States)
| home_state1 = Virginia
| electoral_vote1 = 69
| states_carried1 = 10
| 


{1788: []}

'ab'